<a href="https://colab.research.google.com/github/DrsimpleMugo/Machine-Learning-models/blob/Deployment/FinalCornDeployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask-ngrok

In [ ]:
cd /content/drive/MyDrive/CornDeployment

/content/drive/MyDrive/CornDeployment


In [ ]:
import flask
from flask import Flask, render_template, request
import numpy as np
from flask_ngrok import run_with_ngrok
from keras.preprocessing. image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications import inception_v3
from keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings('ignore')
print('successful')

successful


In [ ]:
from __future__ import division,print_function
import numpy as np
import tensorflow as tf
import sys
import os
import glob
import re

# This block of code is used if you are going to deploy your model in Nvidi GPU if you don't have it then you can skip these codes

from tensorflow.compat.v1 import ConfigProto
from tensorflow.python.keras.backend import set_session


tf.keras.backend.clear_session()
config = tf.compat.v1.ConfigProto(gpu_options = 
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.9)
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

# Upto here 


from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing import image

from flask import Flask,redirect,url_for,request,render_template
from werkzeug.utils import secure_filename


# Define a Flask App
app = Flask(__name__)
run_with_ngrok(app)

# Load model
# graph = tf.get_default_graph()
model_path = 'corn.h5'
model = load_model(model_path)
#model.summary()
#model._make_predict_function()  #Necessary for imagenet weights 
# If model._make_predict_funciton() throws you error then you can discard this line and comment it.


def model_predict(img_path,model):
  img = image.load_img(img_path,target_size=(224,224))
  x = image.img_to_array(img)
  x = np.expand_dims(x,axis=0)
  x = x/255
  x = preprocess_input(x)
  predictions = Dense(4, activation='softmax')(x)
  preds = model.predict(x)
  return preds


@app.route('/',methods = ['GET'])
def index():
	# Main page
	return render_template('./index.html')


@app.route('/',methods=["POST","GET"])
def upload():
  if request.method=='POST':
    f = request.files['file']
    print(f)
    file_path = "/content/drive/MyDrive/CornDeployment/templates/images" + f.filename
    f.save(file_path)
    preds = model_predict(file_path,model)
    print(preds)

    result = np.argmax(preds, axis = 1)

    if result == 0:
      result = 'Corn Blight'
    elif result == 1:
      result = 'Common Rust'
    elif result == 2:
      result = 'Gray Leaf Spot'
    else:
      result = 'Healthy'

    return render_template('./predict.html',result=result)
  else:
    return render_template('./index.html')

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://929ff493b017.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [16/Aug/2021 06:27:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2021 06:27:01] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2021 06:27:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2021 06:27:03] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [16/Aug/2021 06:27:03] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2021 06:42:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2021 06:42:39] "GET /favicon.ico HTTP/1.1" 404 -


<FileStorage: 'Corn_Common_Rust (1256).JPG' ('image/jpeg')>
[[0.00621276 0.98970336 0.00232762 0.00175617]]


127.0.0.1 - - [16/Aug/2021 06:43:17] "POST / HTTP/1.1" 200 -
